In [1]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.utils.data as data_utils
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
from tqdm import tqdm
import timeit

C:\Users\matth\anaconda3\envs\earthquake-prediction\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Input data:  
- CCLE_expression.csv  
Gene expression TPM (transcript per million) values of the protein coding genes for DepMap cell lines  
log2(TPM+1) goes from 0.00 to 17.78


- CRISPR_gene_effect.csv
Gene effect of the protein coding genes for DepMap cell lines
Gene Effect scores derived from CRISPR knockout screens published by Broad’s Achilles and Sanger’s SCORE projects.  
Negative scores imply cell growth inhibition and/or death following gene knockout. Scores are normalized such that nonessential genes have a median score of 0 and independently identified common essentials have a median score of -1.  
SCORE goes from -3.89 to 2.73.


- 9606.protein.info.v11.5.txt


- protein links 9606.protein.links.v11.5.txt




Genes:17386  
Cell Lines:1086  
Primary Diseases:31  
Lineages:28  

## Extract data 

In [2]:
# Extract gene expression from CCLE_expression.csv
df = pd.read_csv('data/CCLE_expression.csv')
gene_expression = df.rename(columns={"Unnamed: 0":"DepMap"}).set_index("DepMap").dropna()

# Extract gene effect from CRISPR_gene_effect.csv
df = pd.read_csv('data/CRISPR_gene_effect.csv')
gene_effect = df.rename(columns={"DepMap_ID":"DepMap"}).set_index("DepMap").dropna()

In [3]:
# Extract protein links 9606.protein.links.v11.5.txt
protein_links = pd.read_csv('data/9606.protein.links.v11.5.txt', sep=' ')

# Extract protein info from 9606.protein.info.v11.5.txt
df = pd.read_csv('data/9606.protein.info.v11.5.txt', sep='	')
protein_info = df.rename(columns={"#string_protein_id":"protein",
                   "preferred_name":"gene_ID"})[["protein","gene_ID"]]

In [4]:
gene_expression.head()

,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),...,H3C2 (8358),H3C3 (8352),AC098582.1 (8916),DUS4L-BCAP29 (115253422),C8orf44-SGK3 (100533105),ELOA3B (728929),NPBWR1 (2831),ELOA3D (100506888),ELOA3 (162699),CDR1 (1038)
DepMap,,,,,,,,,,,,,,,,,,,,,
ACH-001113,4.331992,0.000000,7.364397,2.792855,4.470537,0.028569,1.226509,3.042644,6.499686,4.739848,...,2.689299,0.189034,0.201634,2.130931,0.555816,0.0,0.275007,0.0,0.0,0.000000
ACH-001289,4.566815,0.584963,7.106537,2.543496,3.504620,0.000000,0.189034,3.813525,4.221104,3.481557,...,1.286881,1.049631,0.321928,1.464668,0.632268,0.0,0.014355,0.0,0.0,0.000000
ACH-001339,3.150560,0.000000,7.379032,2.333424,4.227279,0.056584,1.310340,6.687061,3.682573,3.273516,...,0.594549,1.097611,0.831877,2.946731,0.475085,0.0,0.084064,0.0,0.0,0.042644
ACH-001538,5.085340,0.000000,7.154109,2.545968,3.084064,0.000000,5.868143,6.165309,4.489928,3.956986,...,0.214125,0.632268,0.298658,1.641546,0.443607,0.0,0.028569,0.0,0.0,0.000000
ACH-000242,6.729145,0.000000,6.537607,2.456806,3.867896,0.799087,7.208381,5.569856,7.127014,4.568032,...,1.117695,2.358959,0.084064,1.910733,0.000000,0.0,0.464668,0.0,0.0,0.000000


In [5]:
gene_effect.head()

,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),AADAC (13),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
DepMap,,,,,,,,,,,,,,,,,,,,,
ACH-000001,-0.134808,0.059764,-0.008665,-0.003572,-0.106211,-0.008257,0.018711,-0.291985,0.010921,0.064932,...,-0.037619,-0.116524,-0.029331,0.105940,0.147605,-0.119822,0.063387,0.160857,0.058648,-0.316792
ACH-000004,0.081853,-0.056401,-0.106738,-0.014499,0.078209,-0.137562,0.168657,-0.198560,0.133372,0.151300,...,-0.030901,-0.262220,0.136406,0.031327,0.093763,-0.079692,-0.173709,0.153632,0.175627,-0.040869
ACH-000005,-0.094196,-0.014598,0.100426,0.169103,0.032363,-0.148050,0.168931,-0.244777,-0.086871,-0.036037,...,0.039434,-0.336925,-0.095528,-0.035541,-0.035612,-0.040183,-0.165464,0.077343,0.019387,-0.085687
ACH-000007,-0.011544,-0.123189,0.080692,0.061046,-0.013454,-0.016922,-0.029474,-0.206516,-0.063998,0.139288,...,-0.229303,-0.463191,0.061641,0.190301,0.119388,-0.036695,-0.182449,-0.146936,-0.189451,-0.281167
ACH-000009,-0.050782,-0.037466,0.068885,0.090375,0.012634,-0.079339,-0.017808,-0.183192,0.006227,-0.001700,...,-0.157219,-0.318765,0.015761,0.196949,-0.045874,-0.186805,-0.275629,-0.001227,-0.049140,-0.240582


In [6]:
protein_links.head()

,protein1,protein2,combined_score
0,9606.ENSP00000000233,9606.ENSP00000379496,155
1,9606.ENSP00000000233,9606.ENSP00000314067,197
2,9606.ENSP00000000233,9606.ENSP00000263116,222
3,9606.ENSP00000000233,9606.ENSP00000361263,181
4,9606.ENSP00000000233,9606.ENSP00000409666,270


In [7]:
protein_info.head()

,protein,gene_ID
0,9606.ENSP00000000233,ARF5
1,9606.ENSP00000000412,M6PR
2,9606.ENSP00000001008,FKBP4
3,9606.ENSP00000001146,CYP26B1
4,9606.ENSP00000002125,NDUFAF7


## Clean datasets

In [8]:
gene_expression = gene_expression.sort_values(by = 'DepMap').reindex(sorted(gene_expression.columns), axis = 1)
gene_effect = gene_effect.sort_values(by = 'DepMap').reindex(sorted(gene_effect.columns), axis = 1)

In [9]:
def intersection(lst1, lst2):
    return list(set(lst1).intersection(lst2))

In [10]:
# Keep genes in common
gene_expression_genes = gene_expression.columns
gene_effect_genes = gene_effect.columns

intersect_expression_effect = sorted(intersection(gene_expression_genes, gene_effect_genes))
gene_expression = gene_expression[intersect_expression_effect]
gene_effect = gene_effect[intersect_expression_effect]

In [11]:
def get_gene_ID(name):
    split = name.split(' ')
    assert len(split) == 2
    return split[0]

In [12]:
dict_rename = dict((name,get_gene_ID(name)) for name in intersect_expression_effect)
gene_expression = gene_expression.rename(columns=dict_rename)
gene_effect = gene_effect.rename(columns=dict_rename)

In [13]:
protein_list = intersection(list(protein_info['protein']), set(protein_links['protein1']))

protein_info = protein_info[protein_info['protein'].isin(protein_list)]
protein_links = protein_links[protein_links['protein1'].isin(protein_list)]
protein_links = protein_links[protein_links['protein2'].isin(protein_list)]

In [14]:
# Intersection of DepMap proteins and STRING proteins

gene_ID_expr = [get_gene_ID(name) for name in intersect_expression_effect]
gene_ID_list = sorted(intersection(gene_ID_expr, list(protein_info['gene_ID'])))

gene_expression = gene_expression[gene_ID_list]
gene_effect = gene_effect[gene_ID_list]
protein_info = protein_info[protein_info['gene_ID'].isin(gene_ID_list)]

In [15]:
protein_links = protein_links[protein_links['protein1'].isin(list(protein_info['protein']))]
protein_links = protein_links[protein_links['protein2'].isin(list(protein_info['protein']))]

In [16]:
print(gene_expression.shape)
print(gene_effect.shape)
print(protein_info.shape)
print(protein_links.shape)

(1406, 16481)
(1081, 16481)
(16481, 2)
(9928270, 3)


## Save dataframes in files

In [ ]:
gene_expression.to_csv('preprocess/gene_expression.csv')
gene_effect.to_csv('preprocess/gene_effect.csv')

In [18]:
df_to_save = protein_info.sort_values(by='gene_ID')[['protein', 'gene_ID']]
df_to_save['index'] = range(len(df_to_save))

df_to_save = df_to_save[['index', 'protein', 'gene_ID']]
df_to_save.to_csv(r'./preprocess/index_to_protein_and_gene.txt', header=None, index=None, sep=' ', mode='w')

In [19]:
df_to_save = protein_info.sort_values(by='gene_ID')[['protein']]
df_to_save['index'] = range(len(df_to_save))

df_to_save = df_to_save[['index', 'protein']]
df_to_save.to_csv(r'./preprocess/index_to_protein.txt', header=None, index=None, sep=' ', mode='w')

In [20]:
dict_prot_ID = {}
for _, row in df_to_save[['index','protein']].iterrows():
    index, protein = row
    dict_prot_ID[protein] = index

In [21]:
protein_links['combined_score'].min()

150

In [22]:
%%time
dict_gene_to_protein = {}
for _, row in protein_info[['protein','gene_ID']].iterrows():
    protein, gene_idx = row
    dict_gene_to_protein[gene_idx] = protein

CPU times: total: 1.77 s
Wall time: 1.75 s


In [23]:
%%time
assert False   # be careful, young boy
adjacence_matrix = np.zeros((len(protein_info),len(protein_info)))

for index, row in protein_links[['protein1','protein2', 'combined_score']].iterrows():
    protein1, protein2, score = row
    i = dict_prot_ID[protein1]
    j = dict_prot_ID[protein2]
    adjacence_matrix[i,j] = score

AssertionError: 

In [24]:
fig = plt.figure(figsize=(20,20))
plt.imshow(adjacence_matrix[:1000,:1000],cmap='gray') #, cmap="YlGnBu",interpolation='nearest')
plt.title("Adjacence matrix")
plt.show()

NameError: name 'adjacence_matrix' is not defined

<Figure size 1440x1440 with 0 Axes>

In [25]:
%%time
np.savetxt('./preprocess/adjacence_matrix.txt', adjacence_matrix, fmt='%i', delimiter=' ')

NameError: name 'adjacence_matrix' is not defined

## Indexing protein links

In [26]:
index_links = protein_links.copy()
index_links['protein1'] = protein_links['protein1'].apply(lambda prot: dict_prot_ID[prot])
index_links['protein2'] = protein_links['protein2'].apply(lambda prot: dict_prot_ID[prot])
index_links=index_links.rename(columns={'protein1':'index1', 'protein2':'index2'})
index_links

,index1,index2,combined_score
0,820,10020,155
1,820,9796,197
2,820,11255,222
3,820,11365,181
4,820,13660,270
...,...,...,...
11938261,4264,8507,408
11938262,4264,3532,160
11938264,4264,10588,256
11938265,4264,2732,281


In [27]:
edge_index = index_links[['index1', 'index2']].to_numpy().T
edge_index.shape

(2, 9928270)

In [28]:
edge_attr = index_links['combined_score'].to_numpy()
edge_attr.shape

(9928270,)

In [29]:
cell_line_idx = 0
x_expression = gene_expression.iloc[cell_line_idx].to_numpy()
x_effect = gene_effect.iloc[cell_line_idx].to_numpy()
print(x_expression.shape, x_effect.shape)

(16481,) (16481,)


In [50]:
from torch_geometric.data import Data

data = Data(x=torch.LongTensor(x_expression), edge_index=torch.LongTensor(edge_index), edge_attr=torch.LongTensor(edge_attr))

In [51]:
data

Data(x=[16481], edge_index=[2, 9928270], edge_attr=[9928270])

In [52]:
device = torch.device('cuda')
data = data.to(device)

In [54]:
from torch.nn import Linear, ReLU
from torch_geometric.nn import Sequential, GCNConv, GraphConv

In [55]:
graphconv = GraphConv(in_channels = 1, out_channels = 1, aggr = 'mean', bias = True)

In [58]:
#data.num_nodes = len(data.x)
data.validate()

True

In [64]:
data.has_isolated_nodes()

False

In [65]:
data.has_self_loops()

False

In [66]:
data.is_directed()

False

In [68]:
data.keys

['x', 'edge_index', 'edge_attr']

In [69]:
data.num_edges

9928270

In [70]:
data.num_nodes

16481

In [71]:
x_avg = graphconv(x = data.x, edge_index = data.edge_index)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [ ]:
gene_effect.shape

In [ ]:
import sys

In [ ]:
sys.getsizeof(adjacence_matrix)

In [ ]:
sys.getsizeof(protein_links)

In [ ]:
protein_adjacence_list = {}
last_index = 0
total_nb_index = protein_links.shape[0]   # 11938498

In [ ]:
remaining_laps = 1_000_000
for index, row in tqdm(protein_links.iterrows()):
    if index <= last_index:
        continue
    protein1, protein2, score = row
    if (index+1)%(total_nb_index//100) == 0:
        print(f"{(index+1)/total_nb_index *100:.0f} % analyzed")
    if protein1 not in protein_adjacence_list:
        protein_adjacence_list[protein1] = {}
        protein_adjacence_list[protein1][protein2] = score
    last_index = index
    
    remaining_laps -= 1
    if remaining_laps <= 0:
        break
    

In [ ]:
print(protein_links.shape[0])

In [ ]:
print('# possible combinations = %d \n# PPIs = %d' %(16481*16482 / 2, protein_links.shape[0]))

# We see not all proteins are connected ==> # PPIs < # possible combinations

In [ ]:
#protein_adjacence_list['9606.ENSP00000000412']['9606.ENSP00000011653']

In [ ]:
protein_adjacence_list

In [ ]:
def find_neighbors(protein, scan_limit=1_000_000):
    neighbors = {}
    remaining_laps = scan_limit
    for index, (protein1, protein2, score) in protein_links.iterrows():
        if protein1 == protein:
            neighbors[protein2] = score
        if protein2 == protein:
            neighbors[protein1] = score
        
        #if (index+1)%(total_nb_index//100) == 0:
        #    print(f"{(index+1)/total_nb_index *100:.0f} % analyzed")
        
        remaining_laps-=1
        if remaining_laps <= 0:
            break
    
    return neighbors

In [ ]:
neighbors = find_neighbors("9606.ENSP00000005260", scan_limit=1_000_000)

In [ ]:
len(neighbors)

In [ ]:
neighbors

In [ ]:
len(gene_effect.columns)

Personal roadmap:

Save the graph as npy  
Smooth the graph using:
$$new x_* = \lambda x_* + (1-\lambda)\sum_i \frac {\alpha_i} {\sum_j \alpha_j} x_i$$
Then do linear regression

Goal:
plot graph expr vs. graph effect before smoothing  
plot graph expr vs. graph effect after smoothing  (different values of lambda)  


## GraphNN

**Aymeric check this out:**

Graphs from pytorch library (PyG) take COO matrix as an input (sparse version of adjacency matrix) which is $[2 \times N_{edges}]$. You have examples here : https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html
Here's a quick alternative for retrieving edges:

In [ ]:
from tqdm import tqdm
import scipy
from scipy import stats


# row indices of sorted_proteins will serve as node indices
sorted_proteins = protein_info.sort_values(by = 'protein')

# standardize scores of proteins
protein_links_ = protein_links.copy()
protein_links_['combined_score'] = stats.zscore(protein_links_['combined_score'])
protein_links_ = protein_links_[protein_links_['protein1'].isin(sorted_proteins['protein'])]
protein_links_ = protein_links_[protein_links_['protein2'].isin(sorted_proteins['protein'])]

# initializing our COO matrix
N_edges = protein_links_.shape[0] * 2 # multiplied by 2 because graph undirected
edge_index = np.zeros( (2, N_edges) )
edge_attr = np.zeros( (N_edges, 1) )


In [ ]:
max_rows = 1_000_000

for i, row in tqdm(protein_links_[:max_rows].iterrows()):
    protein1, protein2, score = row
    
    print(protein1, protein2)
    
    idx1 = sorted_proteins.index[sorted_proteins['protein'] == protein1].tolist()
    idx2 = sorted_proteins.index[sorted_proteins['protein'] == protein2].tolist()
    
    #assert( len(idx1) == 1 and len(idx2) == 1 )
    
    idx1, idx2 = idx1[0], idx2[0]
    
    # adding both directions to the COO matrix
    edge_index[:, 2*i] = idx1, idx2
    edge_index[:, 2*i + 1] = idx2, idx1
    
    # adding scores to edge_attr matrix 
    edge_attr[2*i:2*i+2, 0] = [score]*2

**Create Data Object for Graph**

In [ ]:
from torch_geometric.data import Data

data = Data(x=x, edge_index=edge_index, ...)